# Lab-2d: Compute the Fibonacci sequence using `for` and `while` loops
A [Fibonacci sequence](https://en.wikipedia.org/wiki/Fibonacci_sequence) is a sequence composed of the `Fibonacci numbers` $F_{n}$ where $F_{n}$ is governed by the recurrence relation:
$$
F_{n} = F_{n-2} + F_{n-1}\quad{n\geq{2}}
$$
where $F_{0}=0$ and $F_{1} = 1$.

### Learning objectives
This lab will familiarize students with working with [Julia methods](https://docs.julialang.org/en/v1/manual/methods/) and using [iteration patterns such as  `for-loops` and `while-loops`](https://docs.julialang.org/en/v1/manual/control-flow/#man-loops) to compute [Fibonacci sequences](https://en.wikipedia.org/wiki/Fibonacci_sequence).
* Let's write two _private methds_ to compute [Fibonacci sequences](https://en.wikipedia.org/wiki/Fibonacci_sequence), one with a `for-loop` and the second with a `while-loop` in the [`Compute.jl` file](src/Compute.jl).
* Then, we'll write a _public function_ that users will automatically call the correct private implementation based on the arguments. Wow! That seems like magic, [but it's just multiple dispatch at work](https://www.youtube.com/watch?v=kc9HwsxE1OY)

Before we begin, break up into teams and take `5 minutes` to review the code files in `src` and then we'll get back together to discuss before we start into the tasks.

## Setup
We set up the computational environment by including the `Include.jl` file using [the `include(...)` method](https://docs.julialang.org/en/v1/base/base/#Base.include). The `Include.jl` file loads external packages, various functions we may need to use in an exercise, custom types to model the components of our example problem, etc.

In [3]:
include("Include.jl");

## Task 1: Build a Fibonacci sequence model (10-min)
In the [Types.jl](/src/Types.jl) file, we formulated [a mutable Julia struct](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that models a [Fibonacci sequence](https://en.wikipedia.org/wiki/Fibonacci_sequence). In particular, the `MyFibonacciSequenceModel` type has the fields:
* `n::Int64` holds the number of elements in the sequence (assuming the sequence is `0`-based). The number of elements in the sequence must be non-negative $n\geq{2}$.
* `sequence::Dict{Int64, Int64}` is a [Julia `Dict` instance]() that holds the sequence values, where the keys of this dictionary are $n$, and the values are the $F_{n}$ values.

With only a few exceptions, we'll always use [a `Factory` software pattern](https://en.wikipedia.org/wiki/Factory_method_pattern) to construct custom composite types, in which we have a specific [`build(...)` method enoded in a Factory.jl file](src/Factory.jl) for each custom type we want to construct. These methods will have two arguments: the type that we want to build and the required data encoded in [`NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple).

In [5]:
model = build(MyFibonacciSequenceModel,(
    n = 50, # pass a value for the sequence length (n ≥ 2). Notice we didn't pass a sequence value ... why?
));
model

MyFibonacciSequenceModel(50, Dict{Int64, Int64}())

__Important concept__: Notice that the [`build(...)` method](src/Factory.jl) uses a [`return` statement](https://docs.julialang.org/en/v1/manual/functions/#The-return-Keyword) at the end of the function body. The [`return` keyword](https://docs.julialang.org/en/v1/manual/functions/#The-return-Keyword), as in many other languages, causes a function to return immediately, providing an expression whose value is returned to the function caller. This is an example of __non-mutating__ function.

### Check: Pass a bad $n$ value to the build method
* __Idea__: Use the _early return pattern_ to check if the argument $n$ is appropriate. If $n<2$, then the `build` function should throw [a `DomainError` instance](https://docs.julialang.org/en/v1/base/base/#Core.DomainError) and never do any computation (is this a good idea?). The [`throws` function](https://docs.julialang.org/en/v1/base/base/#Core.throw) throws an object as an exception, in this case, [an instance of DomainError](https://docs.julialang.org/en/v1/base/base/#Core.DomainError) which is a [Julia built-in error type](https://docs.julialang.org/en/v1/base/base/#Errors). 

In [8]:
let
    # test your bad data handling here ...
    model = build(MyFibonacciSequenceModel,(
        n = -10, # pass a value for the sequence length (n ≥ 2). Notice we didn't pass a sequence value ... why?
    ));
    model
end

LoadError: DomainError with -10:
n must be greater than or equal to 2

In [9]:
model

MyFibonacciSequenceModel(50, Dict{Int64, Int64}())

## Task 2: Develop a Fibonacci for-loop implementation (15 min)
Implement a [private `_fibonacci` method](src/Compute.jl), which takes the `model::MyFibonacciSequenceModel` instance that we created above and a `for-loop` iteration model and computes a [Julia `Dict` instance](https://docs.julialang.org/en/v1/base/collections/#Dictionaries) with entries $n\Rightarrow{F_{n}}$, i.e., the `key` will be the $n$ value and the `value` will be $F_{n}$. The private method should not have a return statement.

The private `_fibonacci` method should be called by the public `fibonacci!` method. Neither the public method nor the private method have return statements. So, how is the data getting returned?
* __Convention__: The public `fibonacci!` function has the bang `!` after the name, marking it as a __mutating function__. This is a naming convention used in the Julia-verse to mark that a method is mutating. Specific [data structures, such as Dict or Arrays](https://en.wikipedia.org/wiki/Data_structure) are mutable. And since arguments are [passed by sharing](https://docs.julialang.org/en/v1/manual/functions/#man-argument-passing), changes made to these [data structures](https://en.wikipedia.org/wiki/Data_structure) in a function, are reflected without returning them.

In [11]:
model |> model -> fibonacci!(model, MyForLoopIterationModel()) # Huh, what is going on here?

Dict{Int64, Int64} with 51 entries:
  5  => 5
  35 => 9227465
  30 => 832040
  32 => 2178309
  6  => 8
  45 => 1134903170
  4  => 3
  13 => 233
  12 => 144
  28 => 317811
  23 => 28657
  41 => 165580141
  43 => 433494437
  11 => 89
  36 => 14930352
  39 => 63245986
  7  => 13
  25 => 75025
  34 => 5702887
  50 => 12586269025
  2  => 1
  10 => 55
  18 => 2584
  26 => 121393
  27 => 196418
  ⋮  => ⋮

### TODO: Benchmark the for-loop implementation
A critical measure of code quality is whether it is correct, i.e., it does what it should. However, assuming correctness, another important metric is performance, both in terms of execution time ([time complexity](https://en.wikipedia.org/wiki/Time_complexity)) and memory usage ([space complexity](https://en.wikipedia.org/wiki/Space_complexity)).
* To measure code performance, we use [tools such as the BenchmarkTools.jl package](https://juliaci.github.io/BenchmarkTools.jl/stable/). This package provides tools to run a particular piece of code multiple times and collects information about the execution time and the memory used.

Let's benchmark the `for-loop` implementation of the `fibonacci` method using [the @benchmark macro exported from the BenchmarkTools.jl package](https://juliaci.github.io/BenchmarkTools.jl/stable/)

In [13]:
let
    iteration_model = MyForLoopIterationModel();
    @benchmark fibonacci!($model, $iteration_model)
end

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  995.900 ns …  9.126 ms  ┊ GC (min … max):  0.00% … 99.95%
 Time  (median):       1.387 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.509 μs ± 91.426 μs  ┊ GC (mean ± σ):  42.58% ±  2.95%

            ▃▅▆█▆▆▂▁▂                                           
  ▂▅▇▄▃▂▂▃▄▇█████████▇▇▅▅▅▃▃▃▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  996 ns          Histogram: frequency by time         2.59 μs <

 Memory estimate: 6.61 KiB, allocs estimate: 10.

## Task 3: Complete the Fibonacci while loop implementation (15 min)
Let's implement another [private `_fibonacci` method](src/Compute.jl), which takes the `model::MyFibonacciSequenceModel` instance that we created above and a `while-loop` iteration model and computes a [Julia `Dict` instance](https://docs.julialang.org/en/v1/base/collections/#Dictionaries) with entries $n\Rightarrow{F_{n}}$, i.e., the `key` will be the $n$ value and the `value` will be $F_{n}$. The private method should not have a return statement.
* Like a `for-loop,` a [while loop](https://docs.julialang.org/en/v1/base/base/#while) has a header line that controls iteration and a body. The same scope rules apply. However, unlike a `for-loop,` a [while loop](https://docs.julialang.org/en/v1/base/base/#while) executes until some condition evaluates to false in the header.

In [15]:
model |> model -> fibonacci!(model, MyWhileLoopIterationModel())

Dict{Int64, Int64} with 51 entries:
  5  => 5
  35 => 9227465
  30 => 832040
  32 => 2178309
  6  => 8
  45 => 1134903170
  4  => 3
  13 => 233
  12 => 144
  28 => 317811
  23 => 28657
  41 => 165580141
  43 => 433494437
  11 => 89
  36 => 14930352
  39 => 63245986
  7  => 13
  25 => 75025
  34 => 5702887
  50 => 12586269025
  2  => 1
  10 => 55
  18 => 2584
  26 => 121393
  27 => 196418
  ⋮  => ⋮

### TODO: Benchmark the while-loop implementation
Let's benchmark the `while-loop` implementation of the `fibonacci` method using [the @benchmark macro exported from the BenchmarkTools.jl package](https://juliaci.github.io/BenchmarkTools.jl/stable/)

In [17]:
let
    iteration_model = MyWhileLoopIterationModel();
    @benchmark fibonacci!($model, $iteration_model)
end

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.000 μs …  9.000 ms  ┊ GC (min … max):  0.00% … 99.95%
 Time  (median):     1.392 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.474 μs ± 90.132 μs  ┊ GC (mean ± σ):  42.14% ±  2.96%

           ▂▃▃█▅▇▇▂▂▃                                         
  ▃█▇▄▂▂▂▃▆███████████▇█▆▅▅▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁ ▃
  1 μs           Histogram: frequency by time        2.53 μs <

 Memory estimate: 6.61 KiB, allocs estimate: 10.